In [19]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from urllib.parse import urljoin

# Function to extract Product Title
def get_title(soup):
    try:
        title = soup.find("span", attrs={"id": 'productTitle'})
        title_value = title.text.strip()
    except AttributeError:
        title_value = ""
    return title_value

# Function to extract Product Price
def get_price(soup):
    try:
        price = soup.find("span", attrs={'id': 'priceblock_ourprice'}).text.strip()
    except AttributeError:
        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id': 'priceblock_dealprice'}).text.strip()
        except AttributeError:
            price = ""
    return price

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id': 'availability'})
        available = available.find("span").text.strip()
    except AttributeError:
        available = "Not Available"
    return available

if __name__ == '__main__':
    # Add your user agent and other headers
    HEADERS = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5',
        'Referer': 'https://www.amazon.in/'
    }

    # The webpage URL
    URL = "https://www.amazon.in/s?k=novels&crid=1LKT5JTZXEV15&sprefix=novel%2Caps%2C383&ref=nb_sb_noss_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class': 'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
        absolute_url = link.get('href')
        full_url = urljoin("https://www.amazon.in", absolute_url)
        links_list.append(full_url)
        time.sleep(2)  # Adding a delay of 2 seconds between requests

    d = {"title": [], "price": [], "availability": []}

    # Loop for extracting product details from each link
    for link in links_list:
        new_webpage = requests.get(link, headers=HEADERS)
        new_soup = BeautifulSoup(new_webpage.content, "html.parser")
        title = get_title(new_soup)
        price = get_price(new_soup)
        availability = get_availability(new_soup)
        print(f"Title: {title}, Price: {price}, Availability: {availability}")
        # Function calls to display all necessary product information
        d['title'].append(title)
        d['price'].append(price)
        d['availability'].append(availability)
        time.sleep(2)  # Adding a delay of 2 seconds between requests

    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)   



#this is the loop which worked once, but I wanted the data of the not available products to be included as well.

Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: The Book of Everlasting Things: A Novel, Price: , Availability: In stock
Title: Iron Flame: THE THRILLING SEQUEL TO THE NUMBER ONE GLOBAL BESTSELLING PHENOMENON FOURTH WING (The Empyrean), Price: , Availability: In stock
Title: The Armour of Light (The Kingsbridge Novels, 5), Price: , Availability: In stock
Title: Tagged for Life Will You Be My Wife?, Price: , Availability: In stock
Title: World’s Greatest Classic (Set of 4 Books), Price: , Availability: In stock
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , Availability: Not Available
Title: , Price: , A

In [20]:
amazon_df

,title,price,availability
3,The Book of Everlasting Things: A Novel,,In stock
4,Iron Flame: THE THRILLING SEQUEL TO THE NUMBER...,,In stock
5,"The Armour of Light (The Kingsbridge Novels, 5)",,In stock
6,Tagged for Life Will You Be My Wife?,,In stock
7,World’s Greatest Classic (Set of 4 Books),,In stock
23,Collection of Shakespeares Greatest Stories Bo...,,In stock
